# Load in Data and Required Packages

In [1]:
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dt
from dash.dependencies import Input, Output, State
import inspect
import pandas as pd
import numpy as np
import warnings
import dash_bootstrap_components as dbc
import dash_table_experiments as dt
from dash.exceptions import PreventUpdate
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import json
import portalocker
import dash_utils
from pprint import pprint

<ipython-input-1-a7a1dd21a8bc>:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-a7a1dd21a8bc>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-a7a1dd21a8bc>:5: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


In [2]:
import json

data_path = 'questions.json'

# Open and read the JSON file
with open(data_path, 'r') as json_file:
    data = json.load(json_file)
# Create a DataFrame from the JSON data
df = pd.DataFrame(data)
df

,question,solution,answer
0,Question: Brennan is in a 100-mile race. He ha...,def solution():\n #Brennan has 30 of the 10...,245.5
1,Question: Sharon and her friend ate at a resta...,def solution():\n #Sharon ate 30 dumplings\...,1206729
2,Question: A toy car sells for $26. It is 30% o...,def solution():\n # A toy car sells for $26...,2.197802
3,"Question: Finding out that a new, nearby gym h...",def solution():\n #Former gym's members wer...,2347537.333333
4,Question: A spiny lobster might scavenge for 1...,def solution():\n #A lobster can scavenge 2...,210.0
...,...,...,...
581,"Question: The scone cost $4, the muffin cost $...","def solution():\n #The scone cost $4, the m...",24
582,Question: 4 pizzas cost $32. 2 pizzas cost $16...,def solution():\n # 4 pizzas cost $32\n ...,8.0
583,Question: Gina is in a candy store that sells ...,def solution():\n #Gina has 5 dollars\n ...,0.5
584,"Question: For my 18th birthday, my parents gav...",def solution():\n #I got $100 for my 18th b...,1459.374246


## Create dataframe for storing data

In [3]:
#feedback = pd.DataFrame(columns = ['label', 'text'])
#i=0
csv_filename = 'feedback.csv'
feedback = pd.read_csv(csv_filename)
i = len(feedback) + 1 

# Build Dash App

In [4]:
markdown_text = '''
Please read the following question carefully. Please label the question as either being solvable or unsolvable'''

In [5]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [6]:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        html.H1("Math Question Generator Feedback Portal"),    
        
        dcc.Markdown(children = markdown_text, style = {'white-space': 'pre-wrap'}, dangerously_allow_html=True),
        
        html.Div([
            
            html.Button(id='submit', n_clicks=0, children = 'Display Question', style = {'fontsize': '12'}),
            dcc.Store(id='button-clicked', data=0)

        ], style={'width': '12%', 'float': 'left', 'white-space': 'pre-wrap'}),
        
        html.Div([
            html.H2(id = 'question1_label'),
            dcc.Markdown(id='question1'),
            dcc.Markdown(id= 'question1_solution'),
            dcc.Markdown(id = 'question1_answer'),
        ], id = 'question1_box', style={'width': '40%', 'float': 'left', 'font-weight': 'bold', 'font-size': '12px', 'white-space': 'pre-wrap', 'display': 'inline-block'}), 
        
        html.Div(
            [
                html.Label('\n Is this question solvable?', style={'font-weight': 'bold'}),
                dcc.RadioItems(
                    id='selected_question',
                    options=[
                        {'label': 'Yes', 'value': '1'},
                        {'label': 'No', 'value': '0'},
                    ]
                ),
                html.Div(html.Button('Submit Feedback', id='submit-val', n_clicks=0)),
                
                dcc.Markdown(id = 'feedback')
            ],
            style={'width': '100%', 'display': 'none', 'whitespace': 'pre-wrap', 'position': 'absolute', 'left': 0},
            id='standard-container'), 
        
        html.Div([
              dcc.Location(id='url', refresh=False),
              html.Div(id='page-content')
])
    ]
)

@app.callback([Output(component_id="question1",component_property="children"), 
               Output(component_id="question1_solution",component_property="children"), 
              Output(component_id = 'question1_answer', component_property = 'children'), 
               Output(component_id = 'question1_label', component_property = 'children')],
                  [Input(component_id = 'submit', component_property= 'n_clicks')])
def pull_questions(n_clicks):
    global df 
    global i
    if n_clicks>0:
        question1 = df.iloc[i]['question']
        question1 = "Question: \n" + question1
        question1_solution = df.iloc[i]['solution']
        question1_solution = "Solution: \n" + question1_solution
        question1_answer = "Answer: \n" + str(df.iloc[i]['answer'])
        question1_label = 'Question'
        i +=1
        
        return question1, question1_solution, question1_answer, question1_label
        
    if n_clicks==0:
        return '', '', '', ''

@app.callback(
    Output('standard-container', 'style'),
    [Input('submit', 'n_clicks')],
    State('button-clicked', 'data')
)
def show_hide_radioitems(n_clicks, button_clicked):
    if n_clicks > button_clicked:
        return {'display': 'block', 'float': 'left', 'width': '50%'}
    else:
        return {'display': 'none'}

@app.callback([Output('feedback', 'children'),
              Output('submit-val', 'n_clicks')],
              [Input('submit-val', 'n_clicks'),
              Input('selected_question', 'value'),
              Input('question1', 'children'),
              Input('question1_solution', 'children')])

def export_feedback(n_clicks, selected_question, question1, question1_solution):
    if n_clicks > 0:
        global feedback
        global df
        feedback = feedback.append({'text': question1 + " " + question1_solution,
                                                'label': selected_question}, ignore_index=True)
            
        return 'Feedback submitted successfully.', 0
    else:
        return '', 0

@app.callback(Output('url', 'refresh'),
              [Input('submit-val', 'n_clicks')])  

def reset(n_clicks):
  if n_clicks > 0:
    return True # refresh page

if __name__== "__main__":
    app.run_server(mode= 'external', host = "0.0.0.0", debug=True)

Dash app running on http://0.0.0.0:8050/


## Save feedback

In [21]:
csv_filename = 'feedback.csv'
feedback.to_csv(csv_filename, index=False, header = True) 
i = len(feedback) + 1
print(i)

586


In [11]:
feedback
#Note that it has question /n. Will need to parse this out. 

,label,text
0,0,Question: \nQuestion: Brennan is in a 100-mile...
1,1,Question: \nQuestion: Sharon and her friend at...
2,0,Question: \nQuestion: A toy car sells for $26....
3,1,"Question: \nQuestion: Finding out that a new, ..."
4,1,Question: \nQuestion: A spiny lobster might sc...
...,...,...
244,1,Question: \nQuestion: A group of 1670960 cows ...
245,1,Question: \nQuestion: Hiro has 8623715 sushi r...
246,1,Question: \nQuestion: There are 365 days in a ...
247,1,Question: \nQuestion: There are 365 days in a ...


## Format Feedback- Do at end!

In [22]:
for i in range(len(feedback)):
    question = feedback.iloc[i]['text']
    question = question.split("Question: \n")[1].strip()
    feedback.iloc[i]['text'] = question
feedback

,label,text
0,0,Question: Brennan is in a 100-mile race. He ha...
1,1,Question: Sharon and her friend ate at a resta...
2,0,Question: A toy car sells for $26. It is 30% o...
3,1,"Question: Finding out that a new, nearby gym h..."
4,1,Question: A spiny lobster might scavenge for 1...
...,...,...
580,1,"Question: The scone cost $4, the muffin cost $..."
581,1,Question: 4 pizzas cost $32. 2 pizzas cost $16...
582,1,Question: Gina is in a candy store that sells ...
583,0,"Question: For my 18th birthday, my parents gav..."


## Final Save

In [23]:
csv_filename = 'feedback.csv'
feedback.to_csv(csv_filename, index=False, header = True) 